In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 신경망 모델 구성

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### 모델 계층

In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1819,  0.2173,  0.5332, -0.5482, -0.1397, -0.0442, -0.1739,  0.1893,
         -0.5138,  0.1949,  0.0890, -0.7635, -0.0198,  0.1001,  0.8262,  0.4492,
          0.3284, -0.0250, -0.5108,  0.0459],
        [-0.1034,  0.3456,  0.2823, -0.6316, -0.2610, -0.0881, -0.1192, -0.2058,
         -0.3518,  0.1551,  0.0984, -0.2278,  0.4522,  0.3051,  0.6344,  0.0112,
          0.1032, -0.3434, -0.2100, -0.0868],
        [-0.0544,  0.4219,  0.4989, -0.5661, -0.0223, -0.2256, -0.3640, -0.1907,
          0.0176,  0.4788,  0.1622, -0.1295,  0.2994,  0.4589,  0.2210,  0.0197,
          0.0578,  0.0769, -0.1953,  0.0798]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.1819, 0.2173, 0.5332, 0.0000, 0.0000, 0.0000, 0.0000, 0.1893, 0.0000,
         0.1949, 0.0890, 0.0000, 0.0000, 0.1001, 0.8262, 0.4492, 0.3284, 0.0000,
         0.0000, 0.0459],
        [0.0000, 0.3456, 0.2823, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1551, 0.0984, 0.0000, 0.4522, 0.3051, 0.634

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
logits

tensor([[-0.0779,  0.0032,  0.0855,  0.2363, -0.0431,  0.0520,  0.3558,  0.0785,
          0.3618,  0.1093],
        [-0.1794,  0.1015,  0.1435,  0.3760, -0.0483,  0.0395,  0.3554,  0.0654,
          0.2012,  0.3113],
        [-0.0414,  0.0460,  0.0548,  0.2628, -0.1278,  0.0879,  0.2976,  0.0173,
          0.2558,  0.1950]], grad_fn=<AddmmBackward>)

In [20]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0256,  0.0058,  0.0155,  ..., -0.0206,  0.0307,  0.0108],
        [-0.0038, -0.0222,  0.0139,  ...,  0.0216, -0.0292,  0.0212]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0324, -0.0163], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0174,  0.0174,  0.0088,  ...,  0.0234, -0.0035,  0.0228],
        [-0.0429,  0.0288,  0.0056,  ...,  0.0422, -0.0103,  0.0070]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_